In [1]:
import pandas as pd
import hygia as hg
import pickle

# Classes instanciations

NOTE: Please check if the model_path matches your path

In [2]:
pre_process_data = hg.PreProcessData(country="MEXICO")
augment_data = hg.AugmentData(country="MEXICO")
feature_engineering = hg.FeatureEngineering()
annotate_data = hg.AnnotateData()
new_rf_model = hg.RandomForestModel()

running feature engineering with configs below...
language -> es
dimensions -> 25


/home/anapaula/dell/Playground/hygia/data_pipeline/augment_data/augment_data.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_zipcode_df_raw['postal code'] = country_zipcode_df_raw['postal code'].astype(str).str.pad(country_mappings[country]['length'],fillchar='0')


# Load Data

NOTE: Please check if the file_path matches your data

In [3]:
file_path = '../data/tmp/AI_LATA_ADDRESS_MEX_modificado.csv'
df = pd.read_csv(file_path, sep='¨', nrows=None, engine='python')

# Add new columns

1. Concatenate address
2. All features columns:
    - Key Smash
    - Regex
    - Word Embedding

NOTE: Please check if the columns names matches your data

In [4]:
concatened_column_name = 'concat_STREET_ADDRESS_1_STREET_ADDRESS_2'
df = pre_process_data.pre_process_data(df, ['STREET_ADDRESS_1', 'STREET_ADDRESS_2'], concatened_column_name)
df = feature_engineering.extract_features(df, concatened_column_name)

aliases indified: concat_STREET_ADDRESS_1_STREET_ADDRESS_2 -> ['STREET_ADDRESS_1', 'STREET_ADDRESS_2']
handle null values in the column concat_STREET_ADDRESS_1_STREET_ADDRESS_2
extract features from -> concat_STREET_ADDRESS_1_STREET_ADDRESS_2


# Check new columns names

In [5]:
all_features_columns = [col for col in df if col.startswith('feature_ks') or col.startswith('feature_we') or col.startswith('feature_re')]
all_features_columns

['feature_ks_count_sequence_squared_vowels_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_count_sequence_squared_consonants_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_count_sequence_squared_special_characters_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_ratio_of_numeric_digits_squared_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_average_of_char_count_squared_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_0_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_1_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_2_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_3_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_4_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_5_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_6_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_7_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_8_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_9_concat_STREET_A

# Annotate data

In [6]:
key_smash_thresholds = {
    'count_sequence_squared_vowels': 1.00,
    'count_sequence_squared_consonants': 1.999,
    'count_sequence_squared_special_characters': 2.2499,
    'ratio_of_numeric_digits_squared': 2.9,
    'average_of_char_count_squared': 2.78,
}

df = annotate_data.annotate_data(df, concatened_column_name, key_smash_thresholds)
df

running annotate data with configs below...
thresholds -> {'count_sequence_squared_vowels': 1.0, 'count_sequence_squared_consonants': 1.999, 'count_sequence_squared_special_characters': 2.2499, 'ratio_of_numeric_digits_squared': 2.9, 'average_of_char_count_squared': 2.78}
column -> concat_STREET_ADDRESS_1_STREET_ADDRESS_2


,CUSTOMER_NUM,COMPANY_NUM,ADDRESS_SEQ_NUM,ADDRESS_TYPE,STREET_ADDRESS_1,STREET_ADDRESS_2,CITY,CITY_2,STATE,STATE_L,...,feature_re_only_special_characters_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_email_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_url_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_date_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_exactly_invalid_words_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_is_substring_of_column_name_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_only_one_char_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_only_white_spaces_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_empty_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,target
0,101551647,3,7,S,CARRET FED 307 KM 298.8,CHETUMAL- PUERTO JUAREZ,PLAYA DEL CARME,N SOLIDARIDAD,MX,QROO,...,False,False,False,False,False,False,False,False,False,valid
1,103051972,3,93,S,CARRETERA YERBABUENA-,PUENTECILLA KM. 1.7,GUANAJUATO,,MX,GTO,...,False,False,False,False,False,False,False,False,False,valid
2,103051972,3,95,S,CARR IRAPUATO-ABASOLO KM 5,COL PURISIMA DEL JARDIN,IRAPUATO,,MX,GTO,...,False,False,False,False,False,False,False,False,False,valid
3,103317515,23,0,B,PRESA DE LAS VIRGENES 115,DESP. 6 COL IRRIGACION DELEG.,MEXICO,,MX,DF,...,False,False,False,False,False,False,False,False,False,valid
4,103354057,23,0,S,AV RAFAEL NAVAS SIN NUMERO,COL INDUSTRIAL EL CERRILLO,LERMA,,MX,MEX,...,False,False,False,False,False,False,False,False,False,valid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520291,108405112,23,0,S,CORDILLERA DEL MARQUEZ NO 113,,SAN LUIS POTOSI,,MX,SLP,...,False,False,False,False,False,False,False,False,False,valid
2520292,104633149,23,0,S,CALLE DE BEISBOL 147,COL. CHURUBUSCO COUNTRY CLUB,DELEGACION COYO,"ACAN, MEXICO",MX,DF,...,False,False,False,False,False,False,False,False,False,valid
2520293,112184095,23,0,B,"AV. VICENTE GUERRERO 8951 , CO",L. PARTIDO DOBLADO CD.JUAREZ,CHIHUAHUA,,MX,CHIH,...,False,False,False,False,False,False,False,False,False,valid
2520294,108261326,23,0,S,LAGO GARDA 226,AGUA AZUL,NEZAHUALC,OYOTL,MX,MEX,...,False,False,False,False,False,False,False,False,False,valid


In [7]:
df['target'].value_counts()

valid                             2514405
key_smash                            1695
only_special_characters              1291
contains_email                       1048
contains_exactly_the_word_test        667
contains_exactly_the_word_dell        553
only_one_char                         287
only_numbers                          239
empty                                  71
contains_exactly_invalid_words         26
is_substring_of_column_name            12
contains_date                           2
Name: target, dtype: int64

# Experiment: retrain model

In [8]:
scores = new_rf_model.train_and_get_scores(df, concatened_column_name, all_features_columns)
scores

tranning model...
done
get model score...
accuracy -> 0.9937578027465668
precision -> 0.9794238683127572
recall -> 1.0
f1 -> 0.9896049896049895


{'accuracy': 0.9937578027465668,
 'precision': 0.9794238683127572,
 'recall': 1.0,
 'f1': 0.9896049896049895}

# Predict using pre-trained model

In [9]:
df['prediction'] = new_rf_model.predict(df[all_features_columns], concatened_column_name)
df.drop_duplicates(subset=[concatened_column_name])['prediction'].value_counts()

running model...


/home/anapaula/dell/Playground/hygia/data_pipeline/model/random_forest.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features_column_to_normalize] = df[features_column_to_normalize] / column_absolute_maximum
/home/anapaula/dell/Playground/env/lib/python3.8/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.0    1340347
1.0       5774
Name: prediction, dtype: int64

# Save model and predicted data

In [10]:
new_rf_model.export_model('../data/models/RandomForest_Ksmash_WordEmbedding_Regex_Enrichments_Normalization.pkl',
                          '../data/models/normalization_absolutes.csv')

exporting model and normalization absolutes...


In [12]:
df[df['prediction'] == 1][[concatened_column_name, 'prediction']].drop_duplicates(subset=[concatened_column_name]).to_csv('../data/tmp/prediction_RandomForest_Ksmash_WordEmbedding_Regex_Enrichments_Normalization.csv')